In [61]:
# Treating the traceroute as a sequence of steps towards a destination could reveal more nuanced patterns 


import json
import pprint
import os
import pickle

import matplotlib.pyplot as plt
import socket, struct

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend

from sklearn import metrics
from sklearn import datasets, cluster


In [63]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7160522670547193843
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10940095031864716488
physical_device_desc: "device: XLA_CPU device"
]


In [5]:
df=pd.read_parquet('universe.parquet')
ftagl = cluster.FeatureAgglomeration(n_clusters=50)
reduced = ftagl.fit_transform(df[df.columns.difference([header for header in df.columns if not (header.isdigit())])])

In [29]:
with open('RHSRHHRWHWT', 'wb') as handle:
    pickle.dump(ftagl, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [46]:
# Process data for RNN structure
df=df.drop(columns=df.columns.difference([header for header in df.columns if not (header.isdigit())]))
reduced=pd.DataFrame(reduced)
df=pd.concat([df,reduced],axis=1)
del reduced

bd=df.loc[df.Benign == True]
even=len(bd)
md=df.loc[df.Benign == False]
bd=bd.sample(frac=1).reset_index(drop=True)
md=md.sample(frac=1).reset_index(drop=True)
md=md.loc[df.index < even]
ad=pd.concat([bd,md])
del bd
del md

In [47]:
exclude=ad.columns.difference(['indicator','Benign','Dest','Route','Ping','index','traceroute'])
ad=ad.sample(frac=1).reset_index(drop=True)
X=ad[exclude]
Y=ad['Benign']

In [49]:
X.shape

(33500, 55)

## LSTM Keras

In [51]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_dim=55),
# Add another:
layers.Dense(64, activation='relu'),
# Add a softmax layer with 10 output units:
layers.Dense(10, activation='softmax'),
# Add a softmax layer with 10 output units:
layers.Dense(1, activation='sigmoid')])

In [52]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [54]:
#data = np.random.random((1000, 32))
#labels = random_one_hot_labels((1000, 10))

#val_data = np.random.random((100, 32))
#val_labels = random_one_hot_labels((100, 10))
#print(type(val_labels))
#pprint.pprint(val_labels)

model.fit(X.loc[0:0.8*2*even,:], Y.loc[0:0.8*2*even], epochs=100, verbose=1, batch_size=32,
          validation_data=(X.loc[X.index > 0.8*2*even],Y.loc[Y.index > 0.8*2*even]))

Train on 26801 samples, validate on 6699 samples
Epoch 1/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.5279 - acc: 0.7295 - val_loss: 0.5242 - val_acc: 0.7282
Epoch 2/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.5221 - acc: 0.7334 - val_loss: 0.5284 - val_acc: 0.7234
Epoch 3/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.5186 - acc: 0.7378 - val_loss: 0.5215 - val_acc: 0.7316
Epoch 4/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.5155 - acc: 0.7393 - val_loss: 0.5122 - val_acc: 0.7398
Epoch 5/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.5103 - acc: 0.7471 - val_loss: 0.5180 - val_acc: 0.7392
Epoch 6/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.5091 - acc: 0.7493 - val_loss: 0.5052 - val_acc: 0.7432
Epoch 7/100
26801/26801 [==============================] - 1s 49us/step - loss: 0.5028 - acc: 0.7516 - val_loss: 0.5053 - v

26801/26801 [==============================] - 1s 48us/step - loss: 0.3650 - acc: 0.8393 - val_loss: 0.4000 - val_acc: 0.8170
Epoch 60/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.3646 - acc: 0.8386 - val_loss: 0.3962 - val_acc: 0.8259
Epoch 61/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.3641 - acc: 0.8383 - val_loss: 0.3930 - val_acc: 0.8288
Epoch 62/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.3601 - acc: 0.8414 - val_loss: 0.3958 - val_acc: 0.8252
Epoch 63/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.3615 - acc: 0.8407 - val_loss: 0.3955 - val_acc: 0.8225
Epoch 64/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.3598 - acc: 0.8412 - val_loss: 0.4149 - val_acc: 0.8124
Epoch 65/100
26801/26801 [==============================] - 1s 48us/step - loss: 0.3579 - acc: 0.8424 - val_loss: 0.3930 - val_acc: 0.8258
Epoch 66/100
26801/26801 [==============

In [55]:
data = np.random.random((1000, 32))
labels = random_one_hot_labels((1000, 10))

model.evaluate(X.loc[X.index > 0.8*2*even], Y.loc[Y.index > 0.8*2*even], batch_size=32)

6699/6699 [==============================] - 0s 21us/step


[0.39967341550447494, 0.8243021346380647]

In [56]:
result = model.predict(X.loc[X.index > 0.8*2*even], batch_size=32)
print(result.shape)

(6699, 1)


In [57]:
X

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,AveragePing,NumHops,Tail Timeouts,Timeouts,success
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,76.762850,29,0,9,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,72.797345,30,0,1,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,41.708364,24,0,2,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,71.259714,30,7,9,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,48.868958,30,4,6,True
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,75.219700,30,6,10,True
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,23.916895,30,9,11,True
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.000000,1,2,2,True
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,7.283133,17,0,2,True
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,22.208842,30,10,11,True


In [62]:
print(backend)

<module 'tensorflow._api.v1.keras.backend' from '/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/_api/v1/keras/backend/__init__.py'>
